In [1]:
%pip install pillow
import sys
from PIL import Image
sys.modules['Image'] = Image 
%pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=45,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split = .2)
test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split = .2)

training_set = train_datagen.flow_from_directory('/work/networks/sriram88/DATA',
                                       target_size=(300, 300),
                                                batch_size=32,
                                     class_mode='categorical',
                                            subset='training')

validation_set = test_datagen.flow_from_directory('/work/networks/sriram88/DATA',
                                        target_size=(300, 300),
                                                 batch_size=32,
                                      class_mode='categorical',
                                               shuffle = False,
                                           subset='validation')

2023-10-08 15:11:20.664918: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-08 15:11:20.667631: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-08 15:11:20.698328: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-08 15:11:20.698345: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-08 15:11:20.698365: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

Found 2580 images belonging to 2 classes.
Found 644 images belonging to 2 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import keras
##model building
model = Sequential()
#convolutional layer with rectified linear unit activation
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(300,300,3)))
#32 convolution filters used each of size 3x3
#again
model.add(Conv2D(64, (3, 3), activation='relu'))
#64 convolution filters used each of size 3x3
#choose the best features via pooling
model.add(MaxPooling2D(pool_size=(2, 2)))
#randomly turn neurons on and off to improve convergence
model.add(Dropout(0.25))
#flatten since too many dimensions, we only want a classification output
model.add(Flatten())
#fully connected to get all relevant data
model.add(Dense(128, activation='relu'))
#one more dropout for convergence' sake :) 
model.add(Dropout(0.5))
#output a softmax to squash the matrix into output probabilities
model.add(Dense(2, activation='softmax'))
#Adaptive learning rate (adaDelta) is a popular form of gradient descent rivaled only by adam and adagrad
#categorical ce since we have multiple classes (10) 
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 296, 296, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 148, 148, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 148, 148, 64)      0         
                                                                 
 flatten (Flatten)           (None, 1401856)           0         
                                                                 
 dense (Dense)               (None, 128)               179437696 
                                                        

In [ ]:

#model training
model.fit(training_set,
                    steps_per_epoch=len(training_set),
                    epochs=50,
                    validation_data=validation_set,
                    validation_steps = len(validation_set))

Epoch 1/50
81/81 [==============================] - 87s 1s/step - loss: 0.6763 - accuracy: 0.5554 - val_loss: 0.5996 - val_accuracy: 0.7547
Epoch 2/50
81/81 [==============================] - 85s 1s/step - loss: 0.6226 - accuracy: 0.6519 - val_loss: 0.5202 - val_accuracy: 0.9255
Epoch 3/50
81/81 [==============================] - 86s 1s/step - loss: 0.5909 - accuracy: 0.6981 - val_loss: 0.4870 - val_accuracy: 0.8043
Epoch 4/50
81/81 [==============================] - 84s 1s/step - loss: 0.5671 - accuracy: 0.7271 - val_loss: 0.4581 - val_accuracy: 0.7764
Epoch 5/50
81/81 [==============================] - 84s 1s/step - loss: 0.5449 - accuracy: 0.7442 - val_loss: 0.4111 - val_accuracy: 0.8276
Epoch 6/50
81/81 [==============================] - 84s 1s/step - loss: 0.5151 - accuracy: 0.7740 - val_loss: 0.3525 - val_accuracy: 0.9115
Epoch 7/50
81/81 [==============================] - 85s 1s/step - loss: 0.5004 - accuracy: 0.7705 - val_loss: 0.4032 - val_accuracy: 0.7966
Epoch 8/50
81/81 [==

In [ ]:
%pip install scikit-learn
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model.predict_generator(validation_set)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix for CNN')
cm=confusion_matrix(validation_set.classes, y_pred)
print('Classification Report')
target_names = ['NH4CL', 'Others']
print(classification_report(validation_set.classes, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

labels = ['NH\u2084CL', "Others",]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.title('CNN Confusion Matrix')

plt.savefig('cnn.png')
plt.show()

In [ ]:
import tensorflow as tf
resnet_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False
)

x = resnet_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512,activation='relu')(x)
predictions = tf.keras.layers.Dense(
    int(len(training_set.class_indices.keys())),
    activation='softmax'
)(x)

resnet50_model = tf.keras.models.Model(
    inputs= resnet_model.input, 
    outputs=predictions
)

resnet50_model.compile(
    loss='categorical_crossentropy', 
    optimizer=tf.keras.optimizers.SGD(lr=1e-4, momentum=0.9),
    metrics=['accuracy']
)

resnet_history = resnet50_model.fit_generator(
    training_set, 
    epochs=50, 
    shuffle=True, 
    verbose=1,
    
    validation_data=validation_set
)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = resnet50_model.predict_generator(validation_set)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
cm=confusion_matrix(validation_set.classes, y_pred)
print('Classification Report')
target_names = ['NH4CL', 'Others']
print(classification_report(validation_set.classes, y_pred, target_names=target_names))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import numpy as np

labels = ['NH\u2084CL', "Others",]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)

disp.plot(cmap=plt.cm.Blues)
plt.title('ResNet50 Confusion Matrix')

plt.savefig('resnet50.png')
plt.show()